
# NAMD Protein-ligand complex MD Setup tutorial using Jupyter Notebook

This notebook is wrritten by quantaosun@gmail.com in Shanghai, China, 2021. 
This notebook took PDB ID 7L10, the protease of SARS-CoV-2 with an inhibitor, as an example, relative work was firstly published on ACS Cent. Sci. 2021, 7, 3, 467–475, by William L. Jorgensen, Yale University.

**The overall procedure is as below**
0.   PDB structure check and fix with third-party tools or web server.
1.   Installation of Anaconda,pymol, vmd, and openmm, pdbfixer
2.   PDB ID provided by user
3.   Separation of ligand and protein
4.   Topology generation for ligand using LigParGen web server
5.   Combine new ligand pdb and original protein, mannually renumber ligand atom
6.   Generation of ionized.pdb and ionized.psf by VMD, with Charmm force field.
7.   Generation of conf file for NAMD simulation. 
8.   NAMD simulation 


# Download Github repository and create conda env

In [ ]:
git clone https://github.com/quantaosun/NAMD-MD
cd NAMD-MD
conda create -n NAMD-MD python=3.7
conda activate NAMD-MD
(NAMD-MD) conda install jupyter
(NAMD-MD) jupyter notebook
# If you have problem opening jupyter notebook
# conda remove jupyter
# sudo apt isntall jupyter
# jupyter notebook

# Starting Jupyter notebook and install dependencies, this may take a while

In [ ]:
! yes | conda install -c conda-forge vmd &> /dev/null
#@title Install VMD
!yes | conda install -c conda-forge pymol-open-source &> /dev/null
#install openmm
!yes | conda install -c conda-forge openmm &> /dev/null
#install pdbfixer
!yes | conda install -c conda-forge pdbfixer &> /dev/null
#install open babel
!yes|conda install -c openbabel openbabel
#install visualization
! pip install py3Dmol &> /dev/null
#install rdkit
!yes | conda install -c rdkit rdkit &> /dev/null

# Use "conda env list" find your system path

In [ ]:
import sys
sys.path.append('/root/miniconda3/envs/NAMD-MD/lib/python3.7/site-packages')
#import rdkit for visualization purpose
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import Draw
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import DataStructs
from rdkit.Chem import RDConfig
from rdkit.Chem import rdBase
from rdkit.Chem import MolFromPDBFile

# Visually check if your protein has multiple chains or just a single one.

In [ ]:
import py3Dmol
view = py3Dmol.view(query='pdb:7pqs')
view.setStyle({'cartoon':{'color':'spectrum'}})
view

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

# Download PDB 

In [ ]:
PDB_ID = "7PQS" #@param {type:"string"}
pdb = PDB_ID + ".pdb"
! wget https://files.rcsb.org/download/$pdb

--2022-06-07 12:26:45--  https://files.rcsb.org/download/7PQS.pdb
Resolving files.rcsb.org (files.rcsb.org)... 128.6.158.66
Connecting to files.rcsb.org (files.rcsb.org)|128.6.158.66|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/octet-stream]
Saving to: ‘7PQS.pdb’

7PQS.pdb                [   <=>              ] 975.40K  2.01MB/s    in 0.5s    

2022-06-07 12:26:46 (2.01 MB/s) - ‘7PQS.pdb’ saved [998811]



In [ ]:
%cd NAMD-MD

/content/NAMD-MD


In [ ]:
com_file = open('delete.pml','w')
com_file.write('''
load 7PQS.pdb
remove chain B
save chainA.pdb, all 
''')
com_file.close()
#!pymol -c combine_protien_ligand.pml

# (Optional) Skip pdbfixer if your protein has no missing loops

In [ ]:
#Only use this if your protein has a broken backbone
!pdbfixer chainA.pdb --keep-heterogens=none --add-atoms=heavy --ph=7.0 --replace-nonstandard --output=receptor.pdbfixer.pdb

/bin/bash: pdbfixer: command not found


# What is the name of your ligand? Check it out on RCSB PDB website !

In [ ]:
v = py3Dmol.view(query='pdb:7PQS',style={'cartoon':{'colorscheme':'ssPyMol'},'stick':{'radius':0.05}})
v.setStyle({'resn':'80H'},{'stick':{}})
v.zoomTo({'resn':'80H'})

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [ ]:
v = py3Dmol.view('chianA.pdb')
v

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

# Split protein and ligand, input the PDB ID and Ligand ID.

In [ ]:
ligand_name = "80H" #@param {type:"string"}
PDB_path = "/content/NAMD-MD/7PQS.pdb" #@param {type:"string"}
!grep ATOM '{PDB_path}' > protein_no_H.pdb
!grep HETATM '{PDB_path}' > ligand_1.pdb
!grep '{ligand_name}' ligand_1.pdb > ligand_no_H.pdb

# Add hydrogen to ligand

In [ ]:
!obabel -ipdb ligand_no_H.pdb -opdb -O ligand_with_H.pdb -h

/bin/bash: obabel: command not found


# Uplaod ligand_with_H.pdb to LigParGen to generate PDB,PRM and RTF three files.  http://zarbi.chem.yale.edu/ligpargen/index.html 

In [ ]:
com_file = open('combine_protien_ligand.pml','w')
com_file.write('''
load protein_no_H.pdb
load UNK_A73DB4.pdb
select all
save complex.pdb, all 
''')
com_file.close()
!pymol -c combine_protien_ligand.pml

/bin/bash: pymol: command not found


# Modify VMD script and run psfgen

In [ ]:
com_file = open('psfgen.tcl','w')
com_file.write('''
#psfgen for a single-chained protein with a ligand bound
mol delete all
mol load pdb complex.pdb
set bad [atomselect top "resname ACE"]
if {[info exists bad]} {
set chainB [atomselect top "protein and not hydrogen and not resname ACE NME"]
set chainX [atomselect top "residuetype nothing and not resname ACE NME"]
set flag 1
} else {
set chainB [atomselect top "protein and not hydrogen"]
set chainX [atomselect top "residuetype nothing"]
set flag 0}
$chainB writepdb chainB.pdb
$chainX writepdb chainX.pdb
package require psfgen
topology top_all36_prot.rtf
topology UNK_A73DB4.rtf
pdbalias HIS HSD
pdbalias atom SER HG HG1
pdbalias residue HIS HSE
pdbalias atom ILE CD1 CD
if {$flag == 1} {
segment B {
  first ACE
  last CT3
  pdb chainB.pdb
}
} else {
segment B {
  first NONE
  last NONE
  pdb chainB.pdb
}
}
segment X {
  first NONE
  last NONE
  pdb chainX.pdb
}
coordpdb chainB.pdb B
coordpdb chainX.pdb X
guesscoord
regenerate angles dihedrals
writepdb psf-complex.pdb
writepsf psf-complex.psf
exit
''')
com_file.close()
!vmd -dispdev text -e psfgen.tcl

/bin/bash: vmd: command not found


# Solvate the Complex and add ions 

In [ ]:
com_file = open('solvate.tcl','w')
com_file.write('''
package require solvate  
solvate psf-complex.psf psf-complex.pdb -t 15 -o complex_wb 
package require autoionize
autoionize -psf complex_wb.psf -pdb complex_wb.pdb -sc 0.15 -cation SOD -o ionized 
exit
''')
com_file.close()
!vmd -dispdev text -e solvate.tcl

/bin/bash: vmd: command not found


# (Optional, you can skip this) Make restrain on alpha carbon of protein.

In [ ]:
com_file = open('restrain.tcl','w')
com_file.write('''
mol delete all
mol load pdb ionized.pdb
set all [atomselect top "all"]
set sel [atomselect top "protein and name CA"]
$all set beta 0
$sel set beta 1
$all writepdb restrained.pdb
exit
''')
com_file.close()
!vmd -dispdev text -e restrain.tcl

/bin/bash: vmd: command not found


# Measure the water box dimensions

In [ ]:
com_file = open('measure.tcl','w')
com_file.write('''
mol delete all
mol load pdb ionized.pdb
set ubq [atomselect top all]
measure minmax $ubq   
measure center $ubq 
''')
com_file.close()
!vmd -dispdev text -e measure.tcl

/bin/bash: vmd: command not found


In [ ]:
Then up until here, I just want to use ionized.pdb and ionized.psf for input as BFEE2.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Modify NAMD configuration files

# Three parts in nvt.namd need modification, same for npt.namd

In [ ]:
# Periodic Boundary Conditions
cellBasisVector1 ? 0 0
cellBasisVector2 0 ? 0
cellBasisVector3 0 0 ?
cellOrigin ? ? ?
wrapAll on
# PME (for full-system periodic electrostatics)
PME yes
PMEGridSizeX ? ;# product of 2,3,5, slightly bigger than box vector
PMEGridSizeY ? ;# product of 2,3,5
PMEGridSizeZ ? ;# product of 2,3,5

In [ ]:
paraTypeCharmm on

parameters                        top_all36_prot.prm
parameters                        ???????????
parameters                        toppar_water_ions.mod.str

In [ ]:
run ???????? ;# 5000ps

# Run the simulation (This assume you have installed namd)

In [ ]:
!wget https://www.ks.uiuc.edu/Research/namd/alpha/3.0alpha/download/NAMD_3.0alpha8_Linux-x86_64-multicore-CUDA-SingleNode.tar.gz

--2022-06-07 23:51:51--  https://www.ks.uiuc.edu/Research/namd/alpha/3.0alpha/download/NAMD_3.0alpha8_Linux-x86_64-multicore-CUDA-SingleNode.tar.gz
Resolving www.ks.uiuc.edu (www.ks.uiuc.edu)... 130.126.120.43
Connecting to www.ks.uiuc.edu (www.ks.uiuc.edu)|130.126.120.43|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 217898714 (208M) [application/x-gzip]
Saving to: ‘NAMD_3.0alpha8_Linux-x86_64-multicore-CUDA-SingleNode.tar.gz’

NAMD_3.0alpha8_Linu 100%[===================>] 207.80M  86.8MB/s    in 2.4s    

2022-06-07 23:51:57 (86.8 MB/s) - ‘NAMD_3.0alpha8_Linux-x86_64-multicore-CUDA-SingleNode.tar.gz’ saved [217898714/217898714]



In [ ]:
!!tar -xvf NAMD_3.0alpha8_Linux-x86_64-multicore-CUDA-SingleNode.tar.gz

['NAMD_3.0alpha8_Linux-x86_64-multicore-CUDA/',
 'NAMD_3.0alpha8_Linux-x86_64-multicore-CUDA/lib/',
 'NAMD_3.0alpha8_Linux-x86_64-multicore-CUDA/lib/abf/',
 'NAMD_3.0alpha8_Linux-x86_64-multicore-CUDA/lib/abf/zCoord.tcl',
 'NAMD_3.0alpha8_Linux-x86_64-multicore-CUDA/lib/abf/distance-com.tcl',
 'NAMD_3.0alpha8_Linux-x86_64-multicore-CUDA/lib/abf/vectors.tcl',
 'NAMD_3.0alpha8_Linux-x86_64-multicore-CUDA/lib/abf/distance.tcl',
 'NAMD_3.0alpha8_Linux-x86_64-multicore-CUDA/lib/abf/abscissa.tcl',
 'NAMD_3.0alpha8_Linux-x86_64-multicore-CUDA/lib/abf/pkgIndex.tcl',
 'NAMD_3.0alpha8_Linux-x86_64-multicore-CUDA/lib/abf/zCoord-1atom.tcl',
 'NAMD_3.0alpha8_Linux-x86_64-multicore-CUDA/lib/abf/xyDistance.tcl',
 'NAMD_3.0alpha8_Linux-x86_64-multicore-CUDA/lib/abf/abf_script.tcl',
 'NAMD_3.0alpha8_Linux-x86_64-multicore-CUDA/lib/abf/abf.tcl',
 'NAMD_3.0alpha8_Linux-x86_64-multicore-CUDA/lib/replica/',
 'NAMD_3.0alpha8_Linux-x86_64-multicore-CUDA/lib/replica/README.txt',
 'NAMD_3.0alpha8_Linux-x86_64-

In [ ]:

!/content/NAMD-MD/NAMD_3.0alpha8_Linux-x86_64-multicore-CUDA/namd3 +p2 min.namd > min.log
#!namd2 +p2 npt.namd > npt.log


In [ ]:

!/content/NAMD-MD/NAMD_3.0alpha8_Linux-x86_64-multicore-CUDA/namd3 +idlepoll +p2 +devices 0 nvt.namd > nvt.log
#!namd2 +p2 npt.namd > npt.log

In [ ]:

!/content/NAMD-MD/NAMD_3.0alpha8_Linux-x86_64-multicore-CUDA/namd3 +p2 min.namd > min.log
#!namd2 +p2 npt.namd > npt.log

^C


# BFEE2 to generate input for FEP calculation.

In [ ]:
%cd /content/drive/MyDrive/7PQS_Law_library_Alchem/000_eq

/content/drive/MyDrive/7PQS_Law_library_Alchem/000_eq


In [ ]:
!/content/NAMD_3.0alpha8_Linux-x86_64-multicore-CUDA/namd3 +idlepoll +p2 +devices 0 000.1_eq.conf > 000.1_eq.log

FATAL ERROR: UNABLE TO OPEN .psf FILE ../complex.psf
